# Trabajo de Inteligencia Artificial

## Análisis de sentimientos

Realizado por:
- Alicia Sánchez Hossdorf
- Rafael Segura Gómez

Fecha: 26/05/2023

Convocatoria de Junio

<h1> 1. Recopilación de datos. </h1>

In [110]:
import pandas
import numpy

Hemos descargado todos los tweets que necesitamos en un CSV. Por lo que debemos leer los datos del archivo.

In [111]:
tweetsCSV = pandas.read_csv("../Notebook/Tweets/Tweets_entrenamiento.csv", header=0)


Para saber cuantos tweets tenemos, miraremos cuantas filas tiene el archivo.

In [112]:
print(tweetsCSV.shape)

(27480, 4)


In [113]:
# 10 primeras filas
tweetsCSV.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Como solo necesitamos la columna text, que es donde viene los tweets que necesitamos, haremos una seleccion de dicha columna.


In [114]:
tweets = tweetsCSV[['text']]

In [115]:
tweets.head(20)

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
5,http://www.dothebouncy.com/smf - some shameles...
6,2am feedings for the baby are fun when he is a...
7,Soooo high
8,Both of you
9,Journey!? Wow... u just became cooler. hehe....


<h1> 2. Eliminar las palabras que no aportan información </h1>

In [116]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [117]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [118]:
from nltk import download

download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [119]:
sWords = set(stopwords.words('english'))
print(sWords)

{"shouldn't", 'nor', 've', 'an', 'here', 'o', "don't", 'myself', 'll', 'who', 'hers', 'his', 'more', 'herself', "it's", "aren't", 're', 'me', 'these', 's', 'now', 'over', 'be', 'once', "weren't", 'all', 'ma', 'any', 'too', 'couldn', 'had', "hasn't", 'of', 'ain', 'only', 'yours', 'is', 'yourself', "haven't", 'why', 'are', 'that', "should've", 'or', "you'll", 'your', 'own', 'with', "you'd", 'further', 'how', 'not', 'i', "isn't", 'having', 'ours', 'we', 'can', 'they', 'ourselves', 'and', 'them', "that'll", 'haven', "mightn't", 'have', 't', 'before', 'yourselves', 'which', 'for', 'm', 'been', 'don', 'should', 'through', 'than', 'such', 'this', 'most', "she's", 'did', 'out', 'again', 'between', 'wasn', 'shouldn', 'off', 'then', 'but', 'because', 'each', 'up', 'very', 'in', 'her', 'while', 'isn', 'until', 'some', 'no', 'below', 'to', 'he', 'himself', 'other', 'weren', 'where', 'so', "didn't", 'am', 'there', "won't", 'were', 'theirs', "couldn't", "needn't", 'mightn', 'few', 'it', 'at', 'just'

Crear una función clean_text para eliminar las palabras comunes y poco informativas; eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño; y que utilice técnicas de lematización (stemming) y tokenización para reducir las palabras a su forma base

In [120]:
#Funcion para eliminar las stop words y tokenizado.

def delete_stopWords(tweet):
    tweetLow = tweet.lower()
    tweetWords = word_tokenize(tweetMin)
    tweetFiltered = []
        
    for w in tweetWords:
        if w not in sWords:
            tweetFiltered.append(w)
    return tweetFiltered

In [121]:
#Esta funcion es para eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño.

def delete_otherElements(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("http[^\s]+","",tweet)
    tweetFiltered = re.sub(r"[-()\"#/@;:<>{}+=*~|.!?,]", "", tweet)

    return tweetFiltered
    

## Función para limpiar el text

Con los tweets tokenizados, lematizados y sin stop words lo pasamos a lista.

In [122]:
def clean_text(Tweets): 
    res = Tweets.copy()
    i = 0;
    while i<len(tweets):
        tweet_cleaned = []
        clean_t = delete_otherElements(Tweets['text'].get(i))
        tNoStopWords = delete_stopWords(clean_t)
        #Esto es para la lematización de la frase. 
        for w in tNoStopWords:
            if w != "`":
                tweet_cleaned.append(PorterStemmer().stem(w))
        res.at[i,'text'] = tweet_cleaned
        
        i+=1
    return res

In [123]:
tweets_cleaned = clean_text(tweets)
tweets_cleaned.head(20)

NameError: name 'tweetMin' is not defined

In [ ]:
print(tweets_cleaned)

## tagdo de datos

Documentar:
- !pip install textblob

In [ ]:
import os
import csv
from textblob import TextBlob

In [ ]:

def tag_text(text):
    tweet = ' '.join(text)  # Convertir la lista de palabras en una cadena de text
    blob = TextBlob(tweet)
    tag = blob.sentiment.polarity #Entre -1 y 1, de peor a mejor respectivamente
    return tweet, tag

csv_path = 'Tweets/resultados.csv'

with open(csv_path, 'w', newline='',encoding='utf-8-sig') as csv_file: #Forzamos formato utf-8, otro da problemas
    writer = csv.writer(csv_file)
    writer.writerow(['text', 'tag'])
    
    if len(tweets_cleaned) == 0:
        print("ÇNothing to tag, espabila notas.")
        exit()

    for _, row in tweets_cleaned.iterrows(): #Usamos iterrows porque tweets_cleaned es de tipo dataframe
        resultado = tag_text(row['text'])
        writer.writerow(resultado)


In [ ]:
tweets_results = pandas.read_csv("../Notebook/Tweets/resultados.csv", header=0)

In [ ]:
print(tweets_results.shape)

In [ ]:
# 10 primeras filas
tweets_results.head(20)